In [34]:
import os
import pandas as pd
from dotenv import load_dotenv
from tqdm import tqdm

In [20]:
load_dotenv()

True

In [24]:
pg_user = os.getenv("POSTGRES_USER", "root")
pg_password = os.getenv("POSTGRES_PASSWORD", "<PASSWORD>")
pg_host = os.getenv("POSTGRES_HOST", "localhost")
pg_port = os.getenv("POSTGRES_PORT", "5432")
pg_db = os.getenv("POSTGRES_DB", "postgres")
month = os.getenv("MONTH", 1)
year = os.getenv("YEAR", 2021)
target_table = os.getenv("TARGET_TABLE", "yellow_taxi_data")
chunksize = os.getenv("CHUNKSIZE", 100)

In [2]:
dtype = {
    "VendorID": "Int64",
    "passenger_count": "Int64",
    "trip_distance": "float64",
    "RatecodeID": "Int64",
    "store_and_fwd_flag": "string",
    "PULocationID": "Int64",
    "DOLocationID": "Int64",
    "payment_type": "Int64",
    "fare_amount": "float64",
    "extra": "float64",
    "mta_tax": "float64",
    "tip_amount": "float64",
    "tolls_amount": "float64",
    "improvement_surcharge": "float64",
    "total_amount": "float64",
    "congestion_surcharge": "float64"
}

In [3]:
parse_dates = [
    "tpep_pickup_datetime",
    "tpep_dropoff_datetime"
]

In [4]:
prefix = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/"

In [29]:
df_iter = pd.read_csv(
    f'{prefix}/yellow_tripdata_{year}-{int(month):02d}.csv.gz',
    dtype=dtype,
    parse_dates=parse_dates,
    iterator=True,
    chunksize=int(chunksize)
)
len(next(df_iter))

100000

In [30]:
from sqlalchemy import create_engine

In [15]:
engine = create_engine(f'postgresql://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{pg_db}')

In [32]:
print(pd.io.sql.get_schema(next(df_iter), name='yellow_taxi_data', con=engine))


CREATE TABLE yellow_taxi_data (
	"VendorID" BIGINT, 
	tpep_pickup_datetime TIMESTAMP WITHOUT TIME ZONE, 
	tpep_dropoff_datetime TIMESTAMP WITHOUT TIME ZONE, 
	passenger_count BIGINT, 
	trip_distance FLOAT(53), 
	"RatecodeID" BIGINT, 
	store_and_fwd_flag TEXT, 
	"PULocationID" BIGINT, 
	"DOLocationID" BIGINT, 
	payment_type BIGINT, 
	fare_amount FLOAT(53), 
	extra FLOAT(53), 
	mta_tax FLOAT(53), 
	tip_amount FLOAT(53), 
	tolls_amount FLOAT(53), 
	improvement_surcharge FLOAT(53), 
	total_amount FLOAT(53), 
	congestion_surcharge FLOAT(53)
)




In [35]:
first = True
for df in tqdm(df_iter):
    if first:
        df.head(0).to_sql(target_table, engine, if_exists='replace')
        first = False
        print(f"Created table {target_table}!")

    df.to_sql(target_table, engine, if_exists="append")
    print("Inserted chunk: ", len(df))

0it [00:00, ?it/s]

Created table yellow_taxi_data!


1it [00:06,  6.09s/it]

Inserted chunk:  100000


2it [00:11,  5.95s/it]

...

Inserted chunk:  100000


11it [01:06,  6.20s/it]

Inserted chunk:  100000


12it [01:10,  5.87s/it]

Inserted chunk:  69765
